In [8]:
from datetime import datetime
import subprocess as sp
import xarray as xr
import os


username = ''
password = ''

#Variables to download
variables = ['analysed_sst']
var=' '.join(['--variable '+u for u in variables])

#Creating output Folder
ofo = '../../data/satellite/sst_subskin_l4/'
if not os.path.exists(ofo):  
    os.makedirs(ofo)

#Boundaries of the Mediterranean Sea (Cropping the Atlantic)
min_lon = "-5.61"
max_lon = "36.32"
min_lat = "30.12"
max_lat = "46.02"
    
year_start = 2020
year_end = 2022
years = list(range(year_start,year_end+1))

months = list(range(1,12+1))
month_length = [31,28,31,30,31,30,31,31,30,31,30,31]

In [10]:
for year in years:
    for month in months:
        yyyystart = year
        mmstart = month
        ddstart = 1
        hhstart = " 00:00:00"
        yyyyend = year
        mmend = month
        ddend = month_length[month - 1]
        hhend = " 23:00:00"

        # Modifying the '20xx-x-x' type of string into '20xx-0x-0x'.

        pre_datestart = "{0}-{1}-{2} {3}".format(yyyystart, mmstart, ddstart, hhstart)
        dt = datetime.strptime(pre_datestart, "%Y-%m-%d %H:%M:%S")
        datestart = datetime.strftime(dt, "%Y-%m-%d %H:%M:%S")

        pre_dateend = "{0}-{1}-{2} {3}".format(yyyyend, mmend, ddend, hhend)
        dt = datetime.strptime(pre_dateend, "%Y-%m-%d %H:%M:%S")
        dateend = datetime.strftime(dt, "%Y-%m-%d %H:%M:%S")

        # Output filename
        ofi = "SUBSKIN_SST_L4_{}_{}.nc".format(year, month)

        script = """python -m motuclient --motu https://nrt.cmems-du.eu/motu-web/Motu --service-id SST_MED_PHY_SUBSKIN_L4_NRT_010_036-TDS --product-id cmems_obs-sst_med_phy-sst_nrt_diurnal-oi-0.0625deg_PT1H-m --longitude-min {0} --longitude-max {1} --latitude-min {2} --latitude-max {3} --date-min "{4}" --date-max "{5}" {6} --out-dir {7} --out-name {8} --user {9} --pwd {10}
        """.format(
            min_lon, max_lon, min_lat, max_lat, datestart, dateend, var, ofo, ofi, username, password
        )

        output = sp.getoutput(script)
        print("{}/{} downloaded".format(month, year))

        # Removing the remaining patch of atlantic ocean at Cantabrian Sea

        lat_0 = 42.843059
        lon_0 = -0.179951

        file = ofo + ofi
        ds = xr.open_dataset(file)
        ds = ds.where((ds.lat < lat_0) | (ds.lon > lon_0))

        os.remove(file)

        ds.to_netcdf(file)
        print("{}/{} cropped".format(month, year))

1/2020 downloaded


FileNotFoundError: [Errno 2] No such file or directory: '/home/prossello/MHW_article/data/satellite/sst_subskin_l4/SUBSKIN_SST_L4_2020_1.nc'